In [1]:
from google.cloud import bigquery

client = bigquery.Client()

# Định nghĩa project & dataset
PROJECT_ID = "dw-assignment-242"
DATASET_ID = "job_analysis"
CLEANED_TABLE = f"{PROJECT_ID}.{DATASET_ID}.cleaned_table"

In [2]:
# Tạo các bảng Dimension cho các thuộc tính đơn giản
# =======================================
dim_queries = {
    "DIM_COMPANY": f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_COMPANY` AS
        SELECT ROW_NUMBER() OVER() AS company_id, company
        FROM (SELECT DISTINCT company FROM `{CLEANED_TABLE}`);
    """,
    "DIM_ROLE": f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_ROLE` AS
        SELECT ROW_NUMBER() OVER() AS role_id, role
        FROM (SELECT DISTINCT role FROM `{CLEANED_TABLE}`);
    """,
    "DIM_PORTAL": f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_PORTAL` AS
        SELECT ROW_NUMBER() OVER() AS portal_id, portal
        FROM (SELECT DISTINCT portal FROM `{CLEANED_TABLE}`);
    """,
    "DIM_COUNTRY": f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_COUNTRY` AS
        SELECT ROW_NUMBER() OVER() AS country_id, country
        FROM (SELECT DISTINCT country FROM `{CLEANED_TABLE}`);
    """,
    "DIM_TIME_TYPE": f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_TIME_TYPE` AS
        SELECT ROW_NUMBER() OVER() AS time_type_id, time_type
        FROM (SELECT DISTINCT time_type FROM `{CLEANED_TABLE}`);
    """,
    "DIM_WORK_TYPE": f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_WORK_TYPE` AS
        SELECT ROW_NUMBER() OVER() AS work_type_id, work_type
        FROM (SELECT DISTINCT work_type FROM `{CLEANED_TABLE}`);
    """
}

for table_name, query in dim_queries.items():
    query_job = client.query(query)
    query_job.result()
    print(f"Bảng {table_name} đã được tạo.")

Bảng DIM_COMPANY đã được tạo.
Bảng DIM_ROLE đã được tạo.
Bảng DIM_PORTAL đã được tạo.
Bảng DIM_COUNTRY đã được tạo.
Bảng DIM_TIME_TYPE đã được tạo.
Bảng DIM_WORK_TYPE đã được tạo.


In [3]:
# Tạo bảng DIM_SKILL
# =======================================
dim_skill_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_SKILL` AS
WITH all_skills AS (
  SELECT LOWER(TRIM(skill)) AS skill
  FROM `{CLEANED_TABLE}`,
  UNNEST(SPLIT(REGEXP_REPLACE(skills, r"[\[\]']", ""), ',')) AS skill
)
SELECT ROW_NUMBER() OVER() AS skill_id, skill
FROM (SELECT DISTINCT skill FROM all_skills)
WHERE skill IS NOT NULL AND skill != '';
"""
query_job = client.query(dim_skill_query)
query_job.result()
print("Bảng DIM_SKILL đã được tạo.")

Bảng DIM_SKILL đã được tạo.


In [4]:
# Tạo bảng DIM_TIME (Dimension cho thời gian)
# =======================================
dim_time_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.DIM_TIME` AS
WITH dates AS (
  SELECT DISTINCT PARSE_DATE('%d/%m/%Y', posted_date) AS posted_date
  FROM `{CLEANED_TABLE}`
  WHERE posted_date IS NOT NULL
)
SELECT
  ROW_NUMBER() OVER() AS time_id,
  EXTRACT(DAY FROM posted_date) AS day,
  EXTRACT(MONTH FROM posted_date) AS month,
  EXTRACT(YEAR FROM posted_date) AS year
FROM dates;
"""
query_job = client.query(dim_time_query)
query_job.result()
print("Bảng DIM_TIME đã được tạo.")

Bảng DIM_TIME đã được tạo.


In [5]:
# Tạo bảng FACT_JOB_POSTING (Bảng fact)
# =======================================
fact_job_posting_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.FACT_JOB_POSTING` AS
SELECT
    ROW_NUMBER() OVER() AS job_id,
    c.company_id,
    r.role_id,
    p.portal_id,
    co.country_id,
    tt.time_type_id,
    wt.work_type_id,
    t.time_id,
    avg_year_salary,
FROM `{CLEANED_TABLE}` j
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_COMPANY` c ON j.company = c.company
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_ROLE` r ON j.role = r.role
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_PORTAL` p ON j.portal = p.portal
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_COUNTRY` co ON j.country = co.country
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_TIME_TYPE` tt ON j.time_type = tt.time_type
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_WORK_TYPE` wt ON j.work_type = wt.work_type
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_TIME` t
  ON EXTRACT(DAY FROM PARSE_DATE('%d/%m/%Y', j.posted_date)) = t.day
  AND EXTRACT(MONTH FROM PARSE_DATE('%d/%m/%Y', j.posted_date)) = t.month
  AND EXTRACT(YEAR FROM PARSE_DATE('%d/%m/%Y', j.posted_date)) = t.year;
"""
query_job = client.query(fact_job_posting_query)
query_job.result()
print("Bảng FACT_JOBS đã được tạo.")

Bảng FACT_JOBS đã được tạo.


In [6]:
# Tạo bảng BRIDGE_JOB_SKILL (Bridge Table)
# =======================================
fact_job_skill_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.BRIDGE_JOB_SKILL` AS
WITH cleaned_with_id AS (
  SELECT
    ROW_NUMBER() OVER() AS job_id,
    *
  FROM `{CLEANED_TABLE}`
),
job_skills AS (
  SELECT
    cw.job_id,
    LOWER(TRIM(skill)) AS skill
  FROM cleaned_with_id cw,
  UNNEST(SPLIT(REGEXP_REPLACE(cw.skills, r"[\[\]']", ""), ',')) AS skill
)
SELECT
  js.job_id,
  s.skill_id
FROM job_skills js
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DIM_SKILL` s
  ON js.skill = s.skill
WHERE s.skill_id IS NOT NULL;
"""
query_job = client.query(fact_job_skill_query)
query_job.result()
print("Bảng BRIDGE_JOB_SKILL đã được tạo.")

Bảng BRIDGE_JOB_SKILL đã được tạo.


In [7]:
# Kiểm tra dữ liệu trong FACT_JOB_POSTING
# =======================================
query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.FACT_JOB_POSTING` LIMIT 10"
query_job = client.query(query)
results = query_job.result()
for row in results:
    print(dict(row))

{'job_id': 9, 'company_id': 9, 'role_id': 3, 'portal_id': 2, 'country_id': 5, 'time_type_id': 1, 'work_type_id': 1, 'time_id': 9, 'avg_year_salary': 26444.0}
{'job_id': 96, 'company_id': 76, 'role_id': 4, 'portal_id': 1, 'country_id': 6, 'time_type_id': 1, 'work_type_id': 1, 'time_id': 84, 'avg_year_salary': 42500.0}
{'job_id': 241, 'company_id': 170, 'role_id': 3, 'portal_id': 5, 'country_id': 5, 'time_type_id': 1, 'work_type_id': 1, 'time_id': 157, 'avg_year_salary': 47500.0}
{'job_id': 356, 'company_id': 254, 'role_id': 5, 'portal_id': 1, 'country_id': 5, 'time_type_id': 1, 'work_type_id': 1, 'time_id': 56, 'avg_year_salary': 50000.0}
{'job_id': 362, 'company_id': 255, 'role_id': 3, 'portal_id': 2, 'country_id': 5, 'time_type_id': 1, 'work_type_id': 1, 'time_id': 212, 'avg_year_salary': 50325.0}
{'job_id': 422, 'company_id': 281, 'role_id': 5, 'portal_id': 4, 'country_id': 14, 'time_type_id': 1, 'work_type_id': 1, 'time_id': 231, 'avg_year_salary': 50400.0}
{'job_id': 483, 'company_